In [2]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob
import os

In [3]:
gt_data = pd.read_json(f"./images/configurations.json")
len(gt_data)

120

In [4]:
# text_image_0a5a0d82-477d-47e2-a504-861c5952fe98-gemini-output.md
gt_data["gemini-output-file"] = gt_data["image_name"].apply(
    lambda x: "./images/" + x.replace(".png", "") + "-gemini-output.md"
)

# Read the content of each file for the gemini output and make a new column, ignoring missing files
gt_data["gemini-output-raw"] = gt_data["gemini-output-file"].apply(
    lambda x: (open(x, "r").read() if os.path.exists(os.path.join(x)) else None)
)

In [5]:
# drop rows with missing gemini output
gt_data = gt_data.dropna(subset=["gemini-output-raw"])

In [6]:
len(gt_data)

120

In [7]:
def extract_marked_text(text):
    # First try to extract text between **
    parts = text.split("**")
    if len(parts) > 1:
        # Extract the first marked text which is likely the number or description of intersections
        marked_text = parts[1].lower()
        # Map textual numbers to integer values
        number_mapping = {
            "zero": 0,
            "one": 1,
            "two": 2,
            "three": 3,
            "four": 4,
            "five": 5,
            "six": 6,
            "seven": 7,
            "eight": 8,
            "nine": 9,
            "ten": 10,
        }
        # Check if the marked text is a digit and return the integer if true
        if marked_text.isdigit():
            return int(marked_text)
        # Return the corresponding integer if the text is a known number, otherwise return the text
        return number_mapping.get(marked_text, marked_text)
    else:
        # If not found, return a default value indicating no marked text was found
        return "marker_not_found"


gt_data["extracted_number"] = gt_data["gemini-output-raw"].apply(extract_marked_text)

In [8]:
gt_data["extracted_number"].value_counts()

predicted
5    54
2    30
3    30
4     6
Name: count, dtype: int64

In [9]:
cleaned_data = gt_data.copy()
# drop squares column
cleaned_data = cleaned_data.drop(columns=["squares"])

In [10]:
# Ensure both columns are of integer type before comparison
cleaned_data["depth"] = cleaned_data["depth"].astype(int)
cleaned_data["extracted_number"] = cleaned_data["extracted_number"].astype(int)

cleaned_data["is_correct"] = cleaned_data["depth"] == cleaned_data["extracted_number"]
# Calculate accuracy
accuracy = cleaned_data["is_correct"].mean()
print(f"Overall Accuracy: {accuracy * 100:.2f}%")

Overall Accuracy: 80.00%


In [11]:
len(cleaned_data)

120

In [12]:
# Assuming 'cleaned_data' DataFrame has columns 'line_thickness', 'num_intersections', and 'extracted_number'

# Convert line_thickness to an appropriate numeric type if necessary
cleaned_data["line_thickness"] = cleaned_data["line_thickness"].astype(int)

# Calculate accuracy for each thickness
accuracy_by_thickness = cleaned_data.groupby("line_thickness").apply(
    lambda df: (df["is_correct"]).mean()
)

accuracy_by_thickness = 100 * accuracy_by_thickness.round(2)
accuracy_by_thickness

/var/folders/y1/b5wjnkp96_b7cvzncx0zsry40000gn/T/ipykernel_20355/108657126.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  accuracy_by_thickness = cleaned_data.groupby("line_thickness").apply(


line_thickness
2    80.0
3    80.0
4    80.0
dtype: float64

In [13]:
# Convert line_thickness to an appropriate numeric type if necessary
cleaned_data["depth"] = cleaned_data["depth"].astype(int)

# Calculate accuracy for each thickness
accuracy_by_thickness = cleaned_data.groupby("depth").apply(
    lambda df: (df["is_correct"]).mean()
)

accuracy_by_thickness = 100 * accuracy_by_thickness.round(2)
accuracy_by_thickness

/var/folders/y1/b5wjnkp96_b7cvzncx0zsry40000gn/T/ipykernel_20355/1882676781.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  accuracy_by_thickness = cleaned_data.groupby("depth").apply(


depth
2    100.0
3    100.0
4     20.0
5    100.0
dtype: float64

# Export

In [14]:
cleaned_data.drop(["gemini-output-file"], axis=1, inplace=True)
cleaned_data.rename(columns={"gemini-output-raw": "model_output_raw"}, inplace=True)

In [15]:
cleaned_data["Model"] = ["Gemini-1.5-Pro"] * len(cleaned_data)
cleaned_data.to_pickle("./data/gemini-1.5-pro.pkl")

In [16]:
cleaned_data

,image_name,depth,center,initial_size,reduction_factor,line_thickness,padding,model_output_raw,predicted,is_correct,Model
0,nested_squares_depth_2_image_1_thickness_2.png,2,"[-2.2395532434656262, -2.245365217594155]",17.574157,0.75,2,0.75,There are **2** squares in the image. \n,2,True,Gemini-1.5-Pro
1,nested_squares_depth_2_image_1_thickness_3.png,2,"[-2.2395532434656262, -2.245365217594155]",17.574157,0.75,3,0.75,There are **2** squares in the image. \n,2,True,Gemini-1.5-Pro
2,nested_squares_depth_2_image_1_thickness_4.png,2,"[-2.2395532434656262, -2.245365217594155]",17.574157,0.75,4,0.75,There are **2** squares in the image. \n,2,True,Gemini-1.5-Pro
3,nested_squares_depth_2_image_2_thickness_2.png,2,"[4.522003363881829, 1.575293928741691]",12.158126,0.75,2,0.75,There are **two** squares in the image. \n,2,True,Gemini-1.5-Pro
4,nested_squares_depth_2_image_2_thickness_3.png,2,"[4.522003363881829, 1.575293928741691]",12.158126,0.75,3,0.75,There are **two** squares in the image. \n,2,True,Gemini-1.5-Pro
...,...,...,...,...,...,...,...,...,...,...,...
115,nested_squares_depth_5_image_9_thickness_3.png,5,"[4.67396043765117, -3.083874562085932]",10.089593,0.75,3,0.75,There are **5** squares in the image. \n,5,True,Gemini-1.5-Pro
116,nested_squares_depth_5_image_9_thickness_4.png,5,"[4.67396043765117, -3.083874562085932]",10.089593,0.75,4,0.75,There are **5** squares in the image. \n,5,True,Gemini-1.5-Pro
117,nested_squares_depth_5_image_10_thickness_2.png,5,"[1.960921482573098, 0.529244414005689]",8.411051,0.75,2,0.75,There are **5** squares in the image. \n,5,True,Gemini-1.5-Pro
118,nested_squares_depth_5_image_10_thickness_3.png,5,"[1.960921482573098, 0.529244414005689]",8.411051,0.75,3,0.75,There are **5** squares in the image. \n,5,True,Gemini-1.5-Pro
